<a href="https://colab.research.google.com/github/alikhosravi/Spatial-Accessibility/blob/main/OSMNX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install osmnx

In [ ]:
import geopandas as gpd
from google.colab import drive

In [ ]:
drive.mount('/content/drive')
gdb_file= '/content/drive/My Drive/Isfahan/MyGeodatabase.gdb'
Blocks = gpd.read_file('/content/drive/My Drive/Isfahan/Inputs/block.gpkg')
Blocks.drop(['code',	'zone',	'Shape_Area',	'Mahalat',	'Shape_Leng',	'Shape_Ar_1'], inplace=True, axis=1)
Clinics = gpd.read_file(gdb_file,layer='Clinics')
Hospitals = gpd.read_file(gdb_file,layer='Hospitals')
Labs = gpd.read_file(gdb_file,layer='Labs')
Pharmacy = gpd.read_file(gdb_file,layer='Pharmacy')

In [ ]:
G = ox.graph_from_place('Isfahan, Iran', network_type='drive')
G = ox.utils_graph.get_largest_component(G, strongly=True)
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)

In [ ]:
facilities = ['Hospital', 'Clinic', 'Lab', 'Pharmacy']

In [ ]:
Blocks["Nodes"] = ox.distance.nearest_nodes(G, list(Blocks.centroid.x), list(Blocks.centroid.y))

In [ ]:
from shapely.ops import nearest_points
def get_nearest_values(row, other_gdf, point_column='geometry', value_column="geometry"):
    other_points = other_gdf["geometry"].unary_union
    nearest_geoms = nearest_points(row[point_column], other_points)
    nearest_data = other_gdf.loc[other_gdf["geometry"] == nearest_geoms[1]]
    nearest_value = nearest_data[value_column].values[0]
    return nearest_value


In [ ]:
Blocks['centroid'] = Blocks.centroid
Blocks['nearest_Hospital'] = Blocks.apply(get_nearest_values, other_gdf = Hospitals, point_column="centroid", value_column="geometry", axis=1)
Blocks['nearest_Clinic'] = Blocks.apply(get_nearest_values, other_gdf = Clinics, point_column="centroid", value_column="geometry", axis=1)
Blocks['nearest_Lab'] = Blocks.apply(get_nearest_values, other_gdf = Labs, point_column="centroid", value_column="geometry", axis=1)
Blocks['nearest_Pharmacy'] = Blocks.apply(get_nearest_values, other_gdf = Pharmacy, point_column="centroid", value_column="geometry", axis=1)
Blocks["Nearest_Hospital_Node"] = ox.distance.nearest_nodes(G, list(Blocks.nearest_Hospital.apply(lambda p: p.x)), list(Blocks.nearest_Hospital.apply(lambda p: p.y)))
Blocks["Nearest_Clinic_Node"] = ox.distance.nearest_nodes(G, list(Blocks.nearest_Clinic.apply(lambda p: p.x)), list(Blocks.nearest_Clinic.apply(lambda p: p.y)))
Blocks["Nearest_Lab_Node"] = ox.distance.nearest_nodes(G, list(Blocks.nearest_Lab.apply(lambda p: p.x)), list(Blocks.nearest_Lab.apply(lambda p: p.y)))
Blocks["Nearest_Pharmacy_Node"] = ox.distance.nearest_nodes(G, list(Blocks.nearest_Pharmacy.apply(lambda p: p.x)), list(Blocks.nearest_Pharmacy.apply(lambda p: p.y)))

In [ ]:
for service in facilities:
  times = list()
  for block, facility in zip(Blocks["Nodes"],Blocks["Nearest_"+str(service)+"_Node"]):
    times.append(nx.shortest_path_length(G, block, facility, weight='travel_time'))
  Blocks["T2"+str(service[0])]=times